# Was wordt lijst

In [313]:
import pandas as pd
import numpy as np

## Inladen en opschonen

In [314]:
ww = pd.read_excel(
    '../data/5. Was-wordt-lijst-vegetatietypen-en-habitattypen-09-02-2021.xlsx',
    dtype="string"
)

ww = ww[['rVvN','VvN','SBB-code', 'Habitattypen (blauw: geldige aanvulling; rood: voorstel; zie Toelichting)']].rename(
    columns={'Habitattypen (blauw: geldige aanvulling; rood: voorstel; zie Toelichting)':'Habtype', 'SBB-code':'SBB'}
)

/home/jakko/git/veg2hab/.venv/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


### Verwijderen van rijen waarbij zowel VvN en SBB missen

In [315]:
ww = ww[~((ww['SBB'].isna() & ww['VvN'].isna()))]

### Omzetten van rVvN waardes 'Niet overgenomen in Revisie ....' naar NA

In [316]:
mask = ww['rVvN'].str.lower().str.contains("niet")
ww.loc[mask, 'rVvN'] = np.nan

### Rijen met meerdere VvN codes splitsen

In [317]:
print("Vóór ontdubbelen zijn er {} rijen".format(ww.shape[0]))
ww['VvN'] = ww['VvN'].str.split(',')
ww = ww.explode('VvN')

# Split operation messes up the dtype, do set VvN column back to string dtype
ww['VvN'] = ww['VvN'].astype('string')
print("Na ontdubbelen zijn er {} rijen".format(ww.shape[0]))

Vóór ontdubbelen zijn er 1116 rijen
Na ontdubbelen zijn er 1132 rijen


### Verwijderen haakjes uit VvN codes

In [318]:
ww['VvN'] = ww['VvN'].str.replace("[()]", "", regex=True)

## Controle volledigheid typologieen

In [319]:
ww.nunique()

rVvN       906
VvN        829
SBB        771
Habtype    210
dtype: int64

In [320]:
ww[ww[['VvN', 'SBB']].duplicated(keep=False)]

,rVvN,VvN,SBB,Habtype
34,r38Ab01a,37RG02,<NA>,H2160 mits
34,r38Ab01a,37RG03,<NA>,H2160 mits
39,r38Ab01c,37RG03,<NA>,H2160 mits
41,r45RG07,42Aa01e,<NA>,H2180A mits +H9190 mits (alleen loofbos!)
42,r45RG08,42Aa01e,<NA>,H2180A mits +H9190 mits (alleen loofbos!)
46,r45RG01,42RG02,42-d,H2180A mits + H9110m + H9120m + H9190m
47,r45RG06,42RG02,42-d,H9110 mits + H9120 mits
416,r14Bb02,14Bb02,14D-d,H2130B mits
417,r14Bb02,14Bb02,14D-d,H2130B mits + H2330 mits + H6120 mits
469,r14RG08,14RG11,14-m,H2130A mits + H2130B mits


In [321]:
ww.isna().sum()

rVvN        22
VvN        129
SBB        229
Habtype    183
dtype: int64

In [322]:
# Rijen met een waarde voor rVvN hebben NIET altijd een waarde voor VvN of SBB
ww[~ww['rVvN'].isna()].isna().sum()

rVvN         0
VvN        108
SBB        229
Habtype    183
dtype: int64

In [323]:
# Rijen met een waarde voor VvN hebben WEL altijd een waarde voor rVvN, maar niet voor SBB
ww[~ww['VvN'].isna()].isna().sum()

rVvN         1
VvN          0
SBB        229
Habtype    183
dtype: int64

In [324]:
# Rijen met een waarde voor SBB-code hebben WEL (bijna) altijd een waarde voor rVvN, maar niet voor VvN
ww[~ww['SBB'].isna()].isna().sum()

rVvN        22
VvN        129
SBB          0
Habtype      0
dtype: int64

In [325]:
ww[~ww['SBB'].isna() & ww['rVvN'].isna()]

,rVvN,VvN,SBB,Habtype
468,<NA>,14RG10,14-l,H2130Am mits + H2130Bm mits Toedelen tot revis...
1006,<NA>,<NA>,07-a,"Toedelen tot revisie-type, anders H0000"
1019,<NA>,<NA>,08B-b,"Toedelen tot revisie-type, anders H0000"
1050,<NA>,<NA>,11/c,"Toedelen tot revisie-type, anders H9999 H7110A"
1051,<NA>,<NA>,11A-b,meerekenen met vegetatieloos (incl. algenmatte...
1072,<NA>,<NA>,14/f,"Toedelen tot revisie-type, anders H9999"
1073,<NA>,<NA>,14-m,Toedelen tot revisie-type van klasse 14
1074,<NA>,<NA>,15-d,Toedelen tot revisie-type van klasse 14
1117,<NA>,<NA>,19-f,"toedelen tot revisie-type, anders H9999"
1118,<NA>,<NA>,19-h,"toedelen tot revisie-type, anders H0000"


### Exporteren opgeschoonde WasWordt tabel

In [326]:
ww.to_excel('../data/ww_split.xlsx', index=False, header=True)

## SBB naar VvN vertalen

**Vraag:** Is er een 1-op-1 of veel-op-1 relatie tussen SBB en VvN?

**Antwoord:** Nee, maar wel bij benadering.

In [327]:
number_of_VvN_per_SBB = ww.groupby('SBB')['VvN'].nunique()
number_of_VvN_per_SBB.value_counts()

VvN
1    606
0    111
3     24
2     16
4     11
5      2
6      1
Name: count, dtype: int64

In [328]:
sbb_with_multiple_vvn = number_of_VvN_per_SBB[number_of_VvN_per_SBB > 1].index.to_list()

In [329]:
number_of_Habs_per_SBB = ww.groupby('SBB')['Habtype'].nunique()
number_of_Habs_per_SBB.value_counts()

Habtype
1    749
2     15
3      7
Name: count, dtype: int64

In [330]:
sbb_with_multiple_habs = number_of_Habs_per_SBB[number_of_Habs_per_SBB > 1].index.to_list()

Er zijn redelijk wat SBB typen die vertalen naar meer dan één VvN, maar meestal gaat het dan om subassociaties in VvN die vertalen naar hetzelfde habittype:

In [336]:
set(sbb_with_multiple_vvn) - set(sbb_with_multiple_habs)

{'01A1',
 '01A2',
 '01B1a',
 '07A-a',
 '08B1',
 '09-h',
 '10A2',
 '11-e',
 '11B2',
 '14B1d',
 '16-f',
 '16C1a',
 '20A1e',
 '27A1',
 '28A1',
 '28A2',
 '28A3',
 '29A2',
 '30A1',
 '30B1',
 '30C1',
 '30C2',
 '30D1',
 '30D2',
 '31A2',
 '31B2',
 '31B3',
 '31C1',
 '31D1',
 '32A1',
 '32B2',
 '33A3',
 '33A4',
 '33A5',
 '34A1',
 '36A-b',
 '37-d',
 '37B1',
 '40A-b',
 '41A/a',
 '42A/a',
 '43B-c'}

Slechts voor een paar SBB codes zijn er echt verschillende VvN codes, die vertalen naar verschillende habitattypen:

In [331]:
set(sbb_with_multiple_habs).intersection(sbb_with_multiple_vvn)

{'09B2a',
 '11-h',
 '11-j',
 '12B-c',
 '14-c',
 '14D-d',
 '16C-l',
 '17A1',
 '27A2',
 '29A3',
 '31B1',
 '42-d'}

Bovenstaande SBB codes geinspecteerd. Er zijn in ieder geval de volgende dingen aan de hand:

- Een SBB code vertaalt naar verschillende VvN vegetaties met minstens een andere associatie (en daarmee andere habitattypen)
  - 09B2a vertaalt naar `09Aa03b` of `10Ab01`
  - 11-h vertaalt naar `09Aa02` en `11Aa02a`
  - 11-j vertaalt naar `11Aa01` en `11Aa02`
  - 12B-c vertaalt naar `12Ba01` en `12Ba02`
  - 14-c vertaalt naar `14Aa01` en `14Aa02`
  - 14D-d vertaalt naar `14Bb01`, `14Bb02` en `14RG06`
  - 16C-l vertaalt naar `16Bb01`, `16Bc01` en `16Bc02`
  - 42-d vertaalt naar `42RG02` en `41DG02`
- Een SBB code vertaalt naar meerdere subassociaties in VvN, met verschillende habitatcodes
  - 17A1 vertaalt naar `17Aa01` en subassociaties
  - 27A2 vertaalt naar `27Aa02` en subassociaties
  - 29A3 vertaalt naar `29Aa03` en subassociaties
  - 31B1 vertaalt naar `31Ab01` en subassociaties